## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "sonle" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "34.207.183.20" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://34.207.183.20:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://sonle-mlflow-artifacts-remote/0', creation_time=1716628207411, experiment_id='0', last_update_time=1716628207411, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/26 16:02:49 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 's3://sonle-mlflow-artifacts-remote/1/5fd065d161bc4a629f4f0294f5274777/artifacts'


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://sonle-mlflow-artifacts-remote/1', creation_time=1716710569520, experiment_id='1', last_update_time=1716710569520, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://sonle-mlflow-artifacts-remote/0', creation_time=1716628207411, experiment_id='0', last_update_time=1716628207411, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [9]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [11]:
client.search_registered_models()

[]

In [13]:
# run_id = client.list_run_infos(experiment_id='1')[0].run_id
run_id = "5fd065d161bc4a629f4f0294f5274777"

mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/05/26 16:04:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716710641305, current_stage='None', description='', last_updated_timestamp=1716710641305, name='iris-classifier', run_id='5fd065d161bc4a629f4f0294f5274777', run_link='', source='s3://sonle-mlflow-artifacts-remote/1/5fd065d161bc4a629f4f0294f5274777/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>